In [1]:
import os
import sys
sys.path.append(os.getcwd())

from lib.models.model_execution_support import run_on_inputs_with_targets
from lib.models.model_execution_support import run_mod_with_correction_block_on_inputs_with_targets
from lib.models.recurrent_forecaster_with_correction_block import RecurrentForecasterWithCorrectionBlock
from lib.models.recurrent_sigmoid_piecewise_forecaster import RSPForecaster
from lib.models.linear_forecaster import LinearForecaster
from lib.utils.recurrent_contexts_manager import RecurrentContextsManager

import numpy as np
import pandas as pd
import torch

In [2]:
DEVICE = 'cpu'
DTYPE = torch.float32

In [3]:
INP_SIZE = 2

def sw(t: torch.Tensor) -> torch.Tensor:
    return t.unfold(dimension=1, size=INP_SIZE, step=1)


def get_train_test_data_heart_rate(heart_data, train_ratio=0.9):
    N = heart_data.shape[1]
    train_size = int(N * train_ratio)
    test_size = N - train_size
    train_ts = heart_data[:, :train_size]
    test_ts = heart_data[:, train_size:]
    return (
        sw(torch.from_numpy(train_ts[:, :-1])).permute(1, 0, 2).to(DEVICE, dtype=DTYPE),
        torch.from_numpy(train_ts[:, INP_SIZE:]).permute(1, 0).unsqueeze(-1).to(DEVICE, dtype=DTYPE),
        sw(torch.from_numpy(test_ts[:, :-1])).permute(1, 0, 2).to(DEVICE, dtype=DTYPE),
        torch.from_numpy(test_ts[:, INP_SIZE:]).permute(1, 0).unsqueeze(-1).to(DEVICE, dtype=DTYPE),
    )

df = pd.read_csv('heart_rate.csv')
print(df['T4'][890])
heart_data = np.array(df[:891]).transpose(1, 0)
heart_data = heart_data / (heart_data.max() - heart_data.min())
heart_data = heart_data - heart_data.mean()
print(heart_data)
print(heart_data.shape)

62.0
[[ 0.14598108  0.14598108  0.14204635 ...  0.4683017   0.45550151
   0.43827048]
 [ 0.28219521  0.28219521  0.27689336 ...  0.40076932  0.38879281
   0.38256881]
 [-0.30440775 -0.30440775 -0.30484894 ... -0.36155229 -0.35393276
  -0.34546305]
 [-0.32704289 -0.32704289 -0.32704289 ... -0.29516359 -0.26901976
  -0.2757001 ]]
(4, 891)


In [4]:
inp_len_test = 4
num_sequences_test = 9

cm = RecurrentContextsManager()
pred = LinearForecaster(cm, inp_len_test)
inp_test = torch.randn(num_sequences_test, inp_len_test, device=DEVICE, dtype=DTYPE)
out_test = pred(inp_test)

print(out_test.shape)
print(out_test)
print(pred.predictor.weight.shape)

torch.Size([9, 1])
tensor([[-0.0917],
        [ 0.1167],
        [ 0.2316],
        [ 0.7351],
        [-0.5787],
        [ 0.6034],
        [-0.2639],
        [ 1.0282],
        [ 0.9168]], grad_fn=<AddmmBackward0>)
torch.Size([1, 4])


In [ ]:
inp_len_test = 4
hidden_size_test = 8
num_sequences_test = 9

cm = RecurrentContextsManager()
pred = RSPForecaster(cm, inp_len_test, hidden_size_test, normalize_ctx=True)
pred.init_context(num_sequences_test, device=DEVICE)
print(id(cm.get_ctx(pred.cid)))
print(cm.get_ctx(pred.cid))
print(cm.get_ctx(pred.cid).shape)
inp1_test = torch.randn(num_sequences_test, inp_len_test, device=DEVICE, dtype=DTYPE)
prev_errors1_test = torch.randn(num_sequences_test, 1, device=DEVICE, dtype=DTYPE) * 0.1
out1_test = pred(inp_test)
print(id(cm.get_ctx(pred.cid)))
print(cm.get_ctx(pred.cid))
gt1_test = out_test + torch.randn(num_sequences_test, 1, device=DEVICE, dtype=DTYPE) * 0.1
inp2_test = torch.randn(num_sequences_test, inp_len_test, device=DEVICE, dtype=DTYPE)
out2_test = pred(inp2_test)
print(id(cm.get_ctx(pred.cid)))
print(cm.get_ctx(pred.cid))


print(out1_test.shape)
print(out1_test)
print(out2_test.shape)
print(pred.sigm_lin.weight.shape)

139853414737216
tensor([[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]])
torch.Size([9, 8])
> /home/ivan/my_projects/nau_forecasting/lib/models/recurrent_sigmoid_piecewise_forecaster.py(42)_forward()
     40         out = new_ctx
     41         import pdb; pdb.set_trace()
---> 42         if self.normalize_ctx:
     43             out = F.normalize(out, dim=-1)
     44         if self.use_out_linear:

ipdb> list
     37         z = torch.sigmoid(self.sigm_lin(inp))
     38         ctx_cand = self.w_ctx_cand_lin(inp)
     39         new_ctx = (1 - z) * ctx + z * ctx_cand
     40         out = new_ctx
     41         import pdb; pdb.set_trace()
---> 42         if self.nor

In [6]:
train_inp, train_targ, test_inp, test_targ = get_train_test_data_heart_rate(heart_data)
print(train_inp.shape)
print(train_targ.shape)
print(test_inp.shape)
print(test_targ.shape)

HIDDEN_SIZE = 8
cm = RecurrentContextsManager()
pred = RSPForecaster(cm, INP_SIZE, HIDDEN_SIZE)
outs_train = run_on_inputs_with_targets(pred, train_inp, train_targ)
print(outs_train.shape)
outs_test = run_on_inputs_with_targets(pred, test_inp, test_targ)
print(outs_test.shape)

torch.Size([799, 4, 2])
torch.Size([799, 4, 1])
torch.Size([88, 4, 2])
torch.Size([88, 4, 1])
torch.Size([799, 4, 1])
torch.Size([88, 4, 1])


In [7]:
cm = RecurrentContextsManager()
baseline = LinearForecaster(cm, INP_SIZE)
# INP_SIZE + 3 because correction block also takes baseline
# forecast, previous baseline error and previous actual
# error as extra inputs
corrector = RSPForecaster(cm, INP_SIZE + 3, HIDDEN_SIZE)
pred = RecurrentForecasterWithCorrectionBlock(cm, baseline, corrector)
outs_train = run_mod_with_correction_block_on_inputs_with_targets(pred, train_inp, train_targ)
print(outs_train.shape)
outs_test = run_mod_with_correction_block_on_inputs_with_targets(pred, test_inp, test_targ)
print(outs_test.shape)

torch.Size([799, 4, 1])
torch.Size([88, 4, 1])
